In [2]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import pandas as pd
from patsy import dmatrices
import statsmodels.api as sm 

In [3]:
def max_accuracy(accuracies, diagonal = True):
    """
    Return the maximum accuracy for each session.
    """
    # average over folds
    avg = np.mean(accuracies, axis=1)

    acc_max = []
    for i in range(avg.shape[0]): # looping over sessions
        if diagonal:
            max_acc = np.max(avg[i].diagonal())
        else:
            max_acc = np.max(avg[i])
        acc_max.append(max_acc)
    
    return acc_max

## Linear model to investigates the effects of the decoding strategy given the variablity of the ERF's

In [4]:
lbo_file = 'accuracies_LDA_lbo'
lbo = np.load(f'accuracies/{lbo_file}.npy')

propb_file = 'accuracies_LDA_prop'
propb = np.load(f'accuracies/{propb_file}.npy')

lbo_acc = max_accuracy(lbo, diagonal = False)
propb_acc = max_accuracy(propb, diagonal = False)

# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)
# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)

### preparing design matrix ###
strategy = np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])
accuracy = np.array(lbo_acc + propb_acc)
std = np.concatenate([std_sessions, std_sessions])

# create the design matrix
design = pd.DataFrame({'strategy': strategy, 'std': std, 'accuracy': accuracy})

#y, X = dmatrices('accuracy ~ strategy + std + strategy:std', design)
y, X = dmatrices('accuracy ~ strategy:std', design)
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()

/usr/local/share/anaconda3/envs/mne/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               accuracy   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.080
Method:                 Least Squares   F-statistic:                   0.03737
Date:                Sat, 17 Dec 2022   Prob (F-statistic):              0.850
Time:                        18:47:27   Log-Likelihood:                 35.315
No. Observations:                  14   AIC:                            -66.63
Df Residuals:                      12   BIC:                            -65.35
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.6689      0.008     84.507      0.000       0.652       0.686
strategy:std    -0.0661      0.342     -0.193      0.850      -0.811       0.679
==============================================================================
Omnibus:                        5.111   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.078   Jarque-Bera (JB):                1.397
Skew:                          -0.127   Prob(JB):                        0.497
Kurtosis:                       1.473   Cond. No.                         61.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Difference in accuracy between the two strategies

In [5]:
# do a t-test
from scipy import stats
stats.ttest_rel(lbo_acc, propb_acc)

Ttest_relResult(statistic=0.7213417726468947, pvalue=0.49783205364773386)

In [6]:
for i in range(len(lbo_acc)):
    print(f'session {i+1}')
    print(f'lbo: {lbo_acc[i]}')
    print(f'prob: {propb_acc[i]}')
    print('Difference in accuracy: ', propb_acc[i]- lbo_acc[i])

session 1
lbo: 0.6382438817221426
prob: 0.6446809718699119
Difference in accuracy:  0.0064370901477692755
session 2
lbo: 0.6530919945045145
prob: 0.6434490584598517
Difference in accuracy:  -0.00964293604466282
session 3
lbo: 0.6892553867974042
prob: 0.6852926808948178
Difference in accuracy:  -0.003962705902586405
session 4
lbo: 0.6887538324473477
prob: 0.6953585963501145
Difference in accuracy:  0.006604763902766875
session 5
lbo: 0.6579143103024342
prob: 0.6457431767887877
Difference in accuracy:  -0.012171133513646515
session 6
lbo: 0.6824731102847893
prob: 0.6849220375779516
Difference in accuracy:  0.002448927293162262
session 7
lbo: 0.6724183095864511
prob: 0.668452380952381
Difference in accuracy:  -0.003965928634070082


## Cross decoding

In [12]:
file_source = 'cross_decoding_ncv_5'
cross_source = np.load(f'accuracies/{file_source}.npy')

file_sens = 'cross_decoding_sens_ncv_5'
cross_sens = np.load(f'accuracies/{file_sens}.npy')

# list of 7 empty lists
accuracies_source = []
accuracies_sens = []

for i in range(7):
    tmp = cross_source[i, :, :, :]
    for j in range(7):
        accuracies_source.append(np.max(tmp[j, :, :]))

for i in range(7):
    tmp = cross_sens[i, :, :, :]
    for j in range(7):
        accuracies_sens.append(np.max(tmp[j, :, :]))


In [13]:
# do a t-test
from scipy import stats
stats.ttest_rel(accuracies_source, accuracies_sens)

Ttest_relResult(statistic=-1.9453791904818427, pvalue=0.05759885085323554)

In [16]:
print(np.mean(accuracies_sens))
print(np.mean(accuracies_source))

0.6278882356002234
0.6222184099805399


In [15]:



# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)
# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)

std_test = std_sessions.tolist()*7
std_train = [std_sessions[i] for i in range(7) for j in range(7)]


# create the design matrix
design = pd.DataFrame({'std_test': std_test, 'std_train': std_train, 'accuracy': accuracies})

y, X = dmatrices('accuracy ~ std_train + std_test + std_train:std_test', design)
#y, X = dmatrices('accuracy ~ strategy:std', design)
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               accuracy   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.218
Method:                 Least Squares   F-statistic:                     5.451
Date:                Thu, 15 Dec 2022   Prob (F-statistic):            0.00277
Time:                        12:06:29   Log-Likelihood:                 117.45
No. Observations:                  49   AIC:                            -226.9
Df Residuals:                      45   BIC:                            -219.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              4.5849      1.207      3.800      0.000       2.155       7.015
std_train           -114.6061     35.987     -3.185      0.003    -187.088     -42.124
std_test            -116.5259     35.987     -3.238      0.002    -189.008     -44.044
std_train:std_test  3353.8137   1073.317      3.125      0.003    1192.043    5515.585
==============================================================================
Omnibus:                       13.018   Durbin-Watson:                   1.860
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               13.559
Skew:                           1.131   Prob(JB):                      0.00114
Kurtosis:                       4.234   Cond. No.                     3.28e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.28e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# playground aka mess!!!

In [ ]:
props = np.load('./accuracies/accuracies_LDA_props.npy', allow_pickle=True).squeeze() # proportional session

for i in range(len(props)):
    plt.imshow(props[i], cmap='viridis')
    plt.show()